# Model1_base

The purpose of this notebook / model experiment is to setup a baseline with minimum model configuration.

In [1]:
import pandas as pd
from pycaret.classification import *

## import data

In [2]:
df_ts_agg = pd.read_csv("../proData/df_ts_agg.csv")
df_ts_agg = df_ts_agg.set_index("PATIENT_ID")

df_ts_agg

ihd  Age  Gender  Height                        ICUType  \
PATIENT_ID                                                            
132539        0   54  female     NaN                   Surgical ICU   
132540        0   76    male   175.3  Cardiac Surgery Recovery Unit   
132541        0   44  female     NaN                    Medical ICU   
132543        0   68    male   180.3                    Medical ICU   
132545        0   88  female     NaN                    Medical ICU   
...         ...  ...     ...     ...                            ...   
142665        0   70  female     NaN                   Surgical ICU   
142667        0   25    male     NaN                    Medical ICU   
142670        0   44    male     NaN                    Medical ICU   
142671        1   37    male     NaN                    Medical ICU   
142673        0   78  female   157.5                   Surgical ICU   

            Weight_min  Weight_max  Weight_med  GCS_min  GCS_max  ...  \
PATIENT_ID                                                        ...   
132539             NaN         NaN         NaN     14.0     15.0  ...   
132540            76.0        81.6        80.6      3.0     15.0  ...   
132541            56.7        56.7        56.7      5.0      8.0  ...   
132543            84.6        84.6        84.6     14.0     15.0  ...   
132545             NaN         NaN         NaN     15.0     15.0  ...   
...                ...         ...         ...      ...      ...  ...   
142665            87.0        87.0        87.0      3.0     15.0  ...   
142667           166.4       166.4       166.4     15.0     15.0  ...   
142670           109.0       109.0       109.0      3.0      8.0  ...   
142671            87.4        87.4        87.4      3.0      7.0  ...   
142673            70.7        88.8        88.8      3.0     15.0  ...   

            Lactate_med  Cholesterol_min  Cholesterol_max  Cholesterol_med  \
PATIENT_ID                                                                   
132539              NaN              NaN              NaN              NaN   
132540              NaN              NaN              NaN              NaN   
132541              1.3              NaN              NaN              NaN   
132543              NaN              NaN              NaN              NaN   
132545              NaN              NaN              NaN              NaN   
...                 ...              ...              ...              ...   
142665              2.3              NaN              NaN              NaN   
142667              NaN            117.0            117.0            117.0   
142670              NaN              NaN              NaN              NaN   
142671              1.9              NaN              NaN              NaN   
142673              2.6              NaN              NaN              NaN   

            TroponinI_min  TroponinI_max  TroponinI_med  TroponinT_min  \
PATIENT_ID                                                               
132539                NaN            NaN            NaN            NaN   
132540                NaN            NaN            NaN            NaN   
132541                NaN            NaN            NaN            NaN   
132543                NaN            NaN            NaN            NaN   
132545                NaN            NaN            NaN            NaN   
...                   ...            ...            ...            ...   
142665                NaN            NaN            NaN            NaN   
142667                NaN            NaN            NaN            NaN   
142670                NaN            NaN            NaN            NaN   
142671                NaN            NaN            NaN            NaN   
142673                NaN            NaN            NaN            NaN   

            TroponinT_max  TroponinT_med  
PATIENT_ID                                
132539                NaN            NaN  
132540                NaN      

## setup experiment with pycaret

for baseline, minimum setup parameter has been used

In [3]:
exp_physionet = setup(data = df_ts_agg, target = 'ihd', session_id=123,
                      numeric_features = ["GCS_min", "GCS_max"],
                      #normalize = True, 
                      #transformation = True, 
                      #ignore_low_variance = True,
                      #remove_multicollinearity = True, multicollinearity_threshold = 0.9,
                      #fix_imbalance = True
                     ) 

Setup Succesfully Completed!


## compare common classification models

min(Recall, Precision) is used for comparing, as suggested by the original physionet 2012 challenge

In [4]:
compare_models(turbo = False)

with this simple model setup, not even fixing imbalanced label, Naive Bayesian base model offers min(recall, precision) = 0.34

## create Naive Bayesian model

In [4]:
nb = create_model("nb")

## tune Naive Bayesian base model

In [14]:
tuned_nb = tune_model(nb, optimize = "AUC", choose_better = True)

the tuned Naive Bayesian base model offers a slightly increased min(recall, precision) = 0.35

## evaluate the tuned Naive Bayesian base model

In [6]:
evaluate_model(tuned_nb)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## predict label on unseen data, using tuned Naive Bayesian base model

In [8]:
predict_model(tuned_nb)

Model  Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
0  Naive Bayes    0.8185  0.7834  0.3795  0.3539  0.3663  0.2605  0.2607

Age      Height  Weight_min  Weight_max  Weight_med  GCS_min  GCS_max  \
0     74.0  160.000000   89.300000   103.00000  102.600000      3.0     15.0   
1     57.0  169.677895   68.000000    68.00000   68.000000     14.0     15.0   
2     90.0  170.200000   66.900000    69.00000   66.900000     11.0     15.0   
3     43.0  169.677895   81.083572    84.04358   83.243402     15.0     15.0   
4     66.0  169.677895   45.000000    45.00000   45.000000      9.0     11.0   
...    ...         ...         ...         ...         ...      ...      ...   
1196  81.0  157.500000   66.800000    67.50000   67.500000     10.0     14.0   
1197  58.0  169.677895   71.100000    71.60000   71.600000     14.0     15.0   
1198  73.0  169.677895   66.000000    66.00000   66.000000     14.0     15.0   
1199  19.0  169.677895   90.000000    90.00000   90.000000      7.0     14.0   
1200  82.0  152.400000   61.360000    74.20000   74.200000      3.0     15.0   

      GCS_med  HR_min  HR_max  ...  ICUType_Medical ICU  ICUType_Surgical ICU  \
0        11.5    79.0   100.0  ...                  0.0                   0.0   
1        15.0    89.0   161.0  ...                  1.0                   0.0   
2        14.0    50.0   113.0  ...                  0.0                   1.0   
3        15.0    77.0   112.0  ...                  0.0                   1.0   
4        10.0    50.0    81.0  ...                  0.0                   0.0   
...       ...     ...     ...  ...                  ...                   ...   
1196     12.0    69.0   104.0  ...                  0.0                   1.0   
1197     15.0    57.0   126.0  ...                  0.0                   0.0   
1198     15.0   104.0   139.0  ...                  1.0                   0.0   
1199      8.0    83.0   121.0  ...                  0.0                   1.0   
1200     11.0    80.0    93.0  ...                  0.0                   0.0   

      MechVent_min_0  MechVent_max_0  MechVent_max_1  MechVent_med_0.0  \
0                1.0             0.0             1.0               1.0   
1                1.0             1.0             0.0               1.0   
2                1.0             1.0             0.0               1.0   
3                1.0             1.0             0.0               1.0   
4                1.0             0.0             1.0               1.0   
...              ...             ...             ...               ...   
1196             1.0             1.0             0.0               1.0   
1197             1.0             1.0             0.0               1.0   
1198             1.0             1.0             0.0               1.0   
1199             1.0             0.0             1.0               1.0   
1200             1.0             0.0             1.0               1.0   

      MechVent_med_0.5  ihd  Label   Score  
0                  0.0    0      0  0.0000  
1                  0.0    0      1  0.9992  
2                  0.0    1      1  1.0000  
3                  0.0    0      0  0.0000  
4                  0.0    0      0  0.0000  
...                ...  ...    ...     ...  
1196               0.0    1      0  0.0000  
1197               0.0    0      0  0.0000  
1198               0.0    1      0  0.0000  
1199               0.0    0      0  0.0000  
1200               0.0    0      0  0.0000  

[1201 rows x 125 columns]

on the unseen data, min(recall, precision) = 0.35

## explore blend models, both hard and soft voting

In [9]:
blend_hard = blend_models()

In [10]:
blend_soft = blend_models(method = "soft")

neither blend_hard or blend_soft is performing well, most likely because the majority of models is not performing well with issue of imbalanced label

## conclusion

even with the simplest model, with minimum thoughtful consideration, min(recall, precision) = 0.35 is achieved, surpassing the SAPS-1 score ~0.31 (not apple to apple). 
This promises better model performance upon thoughtful consideration including fix imbalance, remove multicollinearity, normalize/tranform variables, etc.

The following ipynb s address such considerations.